<a href="https://colab.research.google.com/github/venezianof/booksum/blob/main/ADK_Learning_tool_multi_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Welcome to Your ADK Adventure - MultiAgents! 🚀

Welcome back, Agent Architect! This notebook dives into the heart of the Google Agent Development Kit (ADK): orchestrating teams of specialized agents to tackle complex, multi-step problems that a single agent cannot handle alone.

By the end of this session, you will be an expert in advanced agentic workflows:

- **SequentialAgent**: You'll learn to chain agents together, creating pipelines where the output of one agent becomes the input for the next.

- **LoopAgent**: You'll build iterative systems where agents can plan, critique, and refine their work until a specific goal is met, making them "perfectionists."

- **ParallelAgent**: You'll unleash efficiency by running multiple agents simultaneously and then synthesizing their collective findings into a single, comprehensive answer.

- **The Router**: You will construct a "master" router agent that intelligently analyzes a user's request and delegates it to the correct agent or workflow.

Let's get this adventure started!

## Author

HI, I'm Qingyue (Annie) Wang, a developer advocate and AI engineer at **Google**, passionate about helping developers build with AI and cloud technologies :)


If you have questions with this notebook, contact me on [LinkedIn](https://www.linkedin.com/in/anniewangtech/) , [X](https://twitter.com/anniewangtech) or email anniewangtech0510@Gmail.com


```

  (\__/)
  (•ㅅ•)
  /づ  📚       Enjoy learning AI Agents :)


```


-------------
### 🎁 🛑 Important Prerequisite: Setup Your Environment! 🛑 🎁
-----------------------------------------------------------------------------

👉 **Get Your API Key HERE**: https://codelabs.developers.google.com/onramp/instructions#0

 -----------------------------------------------------------------------------

```
 ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️
   /\_/\     /\_/\     /\_/\      /\_/\       /\_/\
  ( ^_^ )   ( -.- )   ( >_< )   ( =^.^= )    ( o_o )             
```


## Part 0: Setup & Authentication 🔑

First things first, let's get all our tools ready. This step installs the necessary libraries and securely configures your Google API key so your agents can access the power of Gemini.

In [1]:
!pip install google-adk google-generativeai -q

# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent, SequentialAgent, LoopAgent, ParallelAgent
from google.adk.tools import google_search, ToolContext
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


In [ ]:
# --- Securely Configure Your API Key ---

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Get Your API Key HERE 👉 https://codelabs.developers.google.com/onramp/instructions#0
# Configure the generative AI library with the provided key
genai.configure(api_key=api_key)

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

In [ ]:
# --- A Helper Function to Run Our Agents ---
# We'll use this function throughout the notebook to make running queries easy.

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"

---
## Part 1: Multi-Agent Mayhem - Sequential Workflows 🧠→🤖→🤖

You've mastered single agents and memory. Now for the most advanced topic: making agents **work together in a sequence**.

Some tasks are too complex for one agent. A user might ask, "Find me a great restaurant and then tell me how to get there." This requires two different skills: food recommendation and navigation.

We'll build a system that can handle this by:
1.  Creating a new `transportation_agent` 🚗.
2.  Teaching our `router_agent` 🧠 to recognize these special "combo" requests.
3.  Writing Python code (the "orchestrator") that runs the agents in a sequence, passing the output of the first agent to the second.

```
                    +---------------------+
                    |    User Query 🗣️     |
                    +----------+----------+
                               |
                               v
                    +---------------------+
                    |   Router Agent 🤖    |
                    |  (Classify Request) |
                    +----------+----------+
                               |
          +--------------------+----------------------+
          |                    |                      |
          v                    v                      v
  +----------------+   +--------------------+  +----------------------+
  |  foodie_agent  |   | weekend_guide_agent|  |  day_trip_agent      |
  |  🍣 Food Search |   | 🎉 Event Discovery |  | 🧳 Trip Planner       |
  +----------------+   +--------------------+  +----------------------+
          |
          v
  +----------------------------+            (if combo request)
  |  Restaurant Recommendation |---------------------------+
  |  ex: "Best sushi is at X"  |                           |
  +----------------------------+                           v
                                                        +-----------------------+
                                                        | transportation_agent  |
                                                        | 🚗 Get Directions      |
                                                        +-----------------------+
                                                        | Input: origin, place  |
                                                        | Output: directions    |
                                                        +-----------------------+

Final Output: 🍽️ Recommendation + 🚗 Route Info
```


In [ ]:
# --- Agent Definitions for our Specialist Team ---
# --- Agent Definition ---

day_trip_agent = Agent(
    name="day_trip_agent",
    model="gemini-2.5-flash",
    description="Agent specialized in generating spontaneous full-day itineraries based on mood, interests, and budget.",
    instruction="""
    You are the "Spontaneous Day Trip" Generator 🚗 - a specialized AI assistant that creates engaging full-day itineraries.

    Your Mission:
    Transform a simple mood or interest into a complete day-trip adventure with real-time details, while respecting a budget.

    Guidelines:
    1. **Budget-Aware**: Pay close attention to budget hints like 'cheap', 'affordable', or 'splurge'. Use Google Search to find activities (free museums, parks, paid attractions) that match the user's budget.
    2. **Full-Day Structure**: Create morning, afternoon, and evening activities.
    3. **Real-Time Focus**: Search for current operating hours and special events.
    4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

    RETURN itinerary in MARKDOWN FORMAT with clear time blocks and specific venue names.
    """,
    tools=[google_search]
)

foodie_agent = Agent(
    name="foodie_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are an expert food critic. Your goal is to find the absolute best food, restaurants, or culinary experiences based on a user's request. When you recommend a place, state its name clearly. For example: 'The best sushi is at **Jin Sho**.'"
)

weekend_guide_agent = Agent(
    name="weekend_guide_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are a local events guide. Your task is to find interesting events, concerts, festivals, and activities happening on a specific weekend."
)

transportation_agent = Agent(
    name="transportation_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are a navigation assistant. Given a starting point and a destination, provide clear directions on how to get from the start to the end."
)

# --- The Brain of the Operation: The Router Agent ---
# We update the router's instructions to know about the new 'combo' task.
router_agent = Agent(
    name="router_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
    Do not answer the query yourself, only return the name of the most appropriate choice.

    Available Options:
    - 'foodie_agent': For queries *only* about food, restaurants, or eating.
    - 'weekend_guide_agent': For queries about events, concerts, or activities happening on a specific timeframe like a weekend.
    - 'day_trip_agent': A general planner for any other day trip requests.
    - 'find_and_navigate_combo': Use this for complex queries that ask to *first find a place* and *then get directions* to it.

    Only return the single, most appropriate option's name and nothing else.
    """
)

# We'll create a dictionary of all our individual worker agents
worker_agents = {
    "day_trip_agent": day_trip_agent,
    "foodie_agent": foodie_agent,
    "weekend_guide_agent": weekend_guide_agent,
    "transportation_agent": transportation_agent, # Add the new agent!
}

print("🤖 Agent team assembled for sequential workflows!")

In [ ]:
# --- Let's Test the Sequential Workflow! ---

async def run_sequential_app():
    queries = [
        "I want to eat the best sushi in Palo Alto.", # Should go to foodie_agent
        "Are there any cool outdoor concerts this weekend?", # Should go to weekend_guide_agent
        "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station." # Should trigger the COMBO
    ]

    for query in queries:
        print(f"\n{'='*60}\n🗣️ Processing New Query: '{query}'\n{'='*60}")

        # 1. Ask the Router Agent to choose the right agent or workflow
        router_session = await session_service.create_session(app_name=router_agent.name, user_id=my_user_id)
        print("🧠 Asking the router agent to make a decision...")
        chosen_route = await run_agent_query(router_agent, query, router_session, my_user_id, is_router=True)
        chosen_route = chosen_route.strip().replace("'", "")
        print(f"🚦 Router has selected route: '{chosen_route}'")

        # 2. Execute the chosen route
        if chosen_route == 'find_and_navigate_combo':
            print("\n--- Starting Find and Navigate Combo Workflow ---")

            # STEP 2a: Run the foodie_agent first
            foodie_session = await session_service.create_session(app_name=foodie_agent.name, user_id=my_user_id)
            foodie_response = await run_agent_query(foodie_agent, query, foodie_session, my_user_id)

            # STEP 2b: Extract the destination from the first agent's response
            # (This is a simple regex, a more robust solution might use a structured output format)
            match = re.search(r'\*\*(.*?)\*\*', foodie_response)
            if not match:
                print("🚨 Could not determine the restaurant name from the response.")
                continue
            destination = match.group(1)
            print(f"💡 Extracted Destination: {destination}")

            # STEP 2c: Create a new query and run the transportation_agent
            directions_query = f"Give me directions to {destination} from the Palo Alto Caltrain station."
            print(f"\n🗣️ New Query for Transport Agent: '{directions_query}'")
            transport_session = await session_service.create_session(app_name=transportation_agent.name, user_id=my_user_id)
            await run_agent_query(transportation_agent, directions_query, transport_session, my_user_id)

            print("--- Combo Workflow Complete ---")

        elif chosen_route in worker_agents:
            # This is a simple, single-agent route
            worker_agent = worker_agents[chosen_route]
            worker_session = await session_service.create_session(app_name=worker_agent.name, user_id=my_user_id)
            await run_agent_query(worker_agent, query, worker_session, my_user_id)
        else:
            print(f"🚨 Error: Router chose an unknown route: '{chosen_route}'")

await run_sequential_app()

---
### Part 2 (The ADK Way): Multi-Agent Mayhem with `SequentialAgent` 🧠→⛓️→🤖

You've seen how to manually link agents together with custom Python code. It works, but it can get complicated. Now, let's refactor our workflow to use a powerful, built-in ADK feature designed specifically for this: the **`SequentialAgent`**.

The `SequentialAgent` is a *workflow agent*. It's not powered by an LLM itself; instead, its only job is to execute a list of other agents in a strict, predefined order.

The real magic ✨ is how it passes information. The ADK uses a shared `state` dictionary that each agent in the sequence can read from and write to.

**Our New Workflow:**

1.  **Foodie Agent**: Finds the restaurant and saves the name to `state['destination']`.
2.  **Transportation Agent**: Automatically reads `state['destination']` and uses it to find directions.

This means we no longer need custom Python code to extract text or build new queries! The ADK handles the plumbing for us.

```
+-------------------------------+
|  find_and_navigate_agent 🧭   |
| SequentialAgent:             |
| 1. Find destination          |
| 2. Get directions            |
+---------------+---------------+
                |
     +----------+----------+
     |                     |
     v                     v
+----------------+   +------------------------+
| foodie_agent 🍣 |   | transportation_agent 🚗 |
| Finds place     |   | Uses {destination}     |
| Output: 'Jin Sho'|   | Output: Directions     |
+----------------+   +------------------------+

Final Output: 🍣 Restaurant + 🚗 Route
```

In [ ]:
# --- Agent Definitions for our Specialist Team (Refactored for Sequential Workflow) ---

# ✨ CHANGE 1: We tell foodie_agent to save its output to the shared state.
# Note the new `output_key` and the more specific instruction.
foodie_agent = Agent(
    name="foodie_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are an expert food critic. Your goal is to find the best restaurant based on a user's request.

    When you recommend a place, you must output *only* the name of the establishment and nothing else.
    For example, if the best sushi is at 'Jin Sho', you should output only: Jin Sho
    """,
    output_key="destination"  # ADK will save the agent's final response to state['destination']
)

# ✨ CHANGE 2: We tell transportation_agent to read from the shared state.
# The `{destination}` placeholder is automatically filled by the ADK from the state.
transportation_agent = Agent(
    name="transportation_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are a navigation assistant. Given a destination, provide clear directions.
    The user wants to go to: {destination}.

    Analyze the user's full original query to find their starting point.
    Then, provide clear directions from that starting point to {destination}.
    """,
)

# ✨ CHANGE 3: Define the SequentialAgent to manage the workflow.
# This agent will run foodie_agent, then transportation_agent, in that exact order.
find_and_navigate_agent = SequentialAgent(
    name="find_and_navigate_agent",
    sub_agents=[foodie_agent, transportation_agent],
    description="A workflow that first finds a location and then provides directions to it."
)

weekend_guide_agent = Agent(
    name="weekend_guide_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are a local events guide. Your task is to find interesting events, concerts, festivals, and activities happening on a specific weekend."
)

# --- The Brain of the Operation: The Router Agent ---

# We update the router to know about our new, powerful SequentialAgent.
router_agent = Agent(
    name="router_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
    Do not answer the query yourself, only return the name of the most appropriate choice.

    Available Options:
    - 'foodie_agent': For queries *only* about food, restaurants, or eating.
    - 'weekend_guide_agent': For queries about events, concerts, or activities happening on a specific timeframe like a weekend.
    - 'day_trip_agent': A general planner for any other day trip requests.
    - 'find_and_navigate_agent': Use this for complex queries that ask to *first find a place* and *then get directions* to it.

    Only return the single, most appropriate option's name and nothing else.
    """
)

# We create a dictionary of all our executable agents for easy lookup.
# This now includes our powerful new workflow agent!
worker_agents = {
    "day_trip_agent": day_trip_agent,
    "foodie_agent": foodie_agent,
    "weekend_guide_agent": weekend_guide_agent,
    "find_and_navigate_agent": find_and_navigate_agent, # Add the new sequential agent
}

print("🤖 Agent team assembled with a SequentialAgent workflow!")

In [ ]:
# --- Let's Test the Streamlined Workflow! ---

async def run_sequential_app():
    queries = [
        "I want to eat the best sushi in Palo Alto.", # Should go to foodie_agent
        "Are there any cool outdoor concerts this weekend?", # Should go to weekend_guide_agent
        "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station." # Should trigger the SequentialAgent
    ]

    for query in queries:
        print(f"\n{'='*60}\n🗣️ Processing New Query: '{query}'\n{'='*60}")

        # 1. Ask the Router Agent to choose the right agent or workflow
        router_session = await session_service.create_session(app_name=router_agent.name, user_id=my_user_id)
        print("🧠 Asking the router agent to make a decision...")
        chosen_route = await run_agent_query(router_agent, query, router_session, my_user_id, is_router=True)
        chosen_route = chosen_route.strip().replace("'", "")
        print(f"🚦 Router has selected route: '{chosen_route}'")

        # 2. Execute the chosen route
        # This logic is now much simpler! The SequentialAgent is treated just like any other worker.
        if chosen_route in worker_agents:
            worker_agent = worker_agents[chosen_route]
            print(f"--- Handing off to {worker_agent.name} ---")
            worker_session = await session_service.create_session(app_name=worker_agent.name, user_id=my_user_id)
            await run_agent_query(worker_agent, query, worker_session, my_user_id)
            print(f"--- {worker_agent.name} Complete ---")
        else:
            print(f"🚨 Error: Router chose an unknown route: '{chosen_route}'")

await run_sequential_app()

### Running the Streamlined App

Notice how much simpler the code below is. There is no longer a special `if chosen_route == 'find_and_navigate_combo':` block with custom logic.

The `find_and_navigate_agent` is now a self-contained unit. We can treat it just like any other agent, hand it the query, and trust the `SequentialAgent` to handle all the internal steps. This makes our main application code much cleaner and easier to read.

---
## Iterative Ideas with `LoopAgent` 🧠→🔁→🤖

Sometimes a task isn't a straight line; it's a loop of refinement. A user might ask for a plan, but have constraints that require checking and re-planning. For this, the ADK provides the **`LoopAgent`**.

The `LoopAgent` executes a sequence of sub-agents repeatedly until a condition is met. This is perfect for workflows involving trial and error, like planning a trip with a tight schedule.

**Our New Workflow: The Perfectionist Planner**

1. **Planner Agent**: Proposes an itinerary (e.g., a museum and a restaurant).
2. **Critic Agent**: Checks the plan against a constraint (e.g., "Is the travel time between these two places less than 30 minutes?").
3. **Refiner Agent**: If the critic finds a problem, this agent takes the feedback and creates a new, improved plan. If the critic is happy, it calls a special `exit_loop` tool to stop the process.

The `LoopAgent` manages this cycle, ensuring we don't get stuck in an infinite loop by setting a `max_iterations` limit.

```
+-------------------------------+
|  iterative_planner_agent 🔁   |
| SequentialAgent:              |
| 1. Propose Plan               |
| 2. Refine in loop (≤ 3 times) |
+---------------+---------------+
                |
     +----------+----------+
     |                     |
     v                     v
+----------------+   +-----------------------+
| planner_agent  |   | refinement_loop 🔁     |
| Propose plan   |   | LoopAgent             |
| e.g., Activity +  | 1. Critic (time check) |
| Restaurant       | 2. Refiner (fix/exit)   |
+----------------+   +-----------------------+

Uses shared state: {current_plan}, {criticism}
Exits when: "Plan is feasible..."

```

In [ ]:
# --- Agent Definitions for an Iterative Workflow ---

# A tool to signal that the loop should terminate
COMPLETION_PHRASE = "The plan is feasible and meets all constraints."
def exit_loop(tool_context: ToolContext):
  """Call this function ONLY when the plan is approved, signaling the loop should end."""
  print(f"  [Tool Call] exit_loop triggered by {tool_context.agent_name}")
  tool_context.actions.escalate = True
  return {}

# Agent 1: Proposes an initial plan
planner_agent = Agent(
    name="planner_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction="You are a trip planner. Based on the user's request, propose a single activity and a single restaurant. Output only the names, like: 'Activity: Exploratorium, Restaurant: La Mar'.",
    output_key="current_plan"
)

# Agent 2 (in loop): Critiques the plan
critic_agent = Agent(
    name="critic_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction=f"""You are a logistics expert. Your job is to critique a travel plan. The user has a strict constraint: total travel time must be short.
    Current Plan: {{current_plan}}
    Use your tools to check the travel time between the two locations.
    IF the travel time is over 45 minutes, provide a critique, like: 'This plan is inefficient. Find a restaurant closer to the activity.'
    ELSE, respond with the exact phrase: '{COMPLETION_PHRASE}'""",
    output_key="criticism"
)

# Agent 3 (in loop): Refines the plan or exits
refiner_agent = Agent(
    name="refiner_agent", model="gemini-2.5-flash", tools=[google_search, exit_loop],
    instruction=f"""You are a trip planner, refining a plan based on criticism.
    Original Request: {{session.query}}
    Critique: {{criticism}}
    IF the critique is '{COMPLETION_PHRASE}', you MUST call the 'exit_loop' tool.
    ELSE, generate a NEW plan that addresses the critique. Output only the new plan names, like: 'Activity: de Young Museum, Restaurant: Nopa'.""",
    output_key="current_plan"
)

# ✨ The LoopAgent orchestrates the critique-refine cycle ✨
refinement_loop = LoopAgent(
    name="refinement_loop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=3
)

# ✨ The SequentialAgent puts it all together ✨
iterative_planner_agent = SequentialAgent(
    name="iterative_planner_agent",
    sub_agents=[planner_agent, refinement_loop],
    description="A workflow that iteratively plans and refines a trip to meet constraints."
)

print("🤖 Agent team updated with an iterative LoopAgent workflow!")

---
## Parallel Power with `ParallelAgent` 🧠→⚡️→🤖🤖🤖

What if a user wants to find multiple, unrelated things at once? "Find me a museum, a concert, AND a restaurant." Running these searches one by one is slow and inefficient.

Enter the **`ParallelAgent`**. This workflow agent executes a list of sub-agents *concurrently*, dramatically speeding up tasks that can be performed independently.

**Our New Workflow: The Multi-Researcher**

1.  **Parallel Agent**: Simultaneously runs three specialist agents:
    - `MuseumFinderAgent`: Finds a museum.
    - `ConcertFinderAgent`: Finds a concert.
    - `FoodieAgent`: Finds a restaurant.
2.  **Synthesis Agent**: Once all three parallel searches are complete, this final agent gathers the results (which were saved to the shared `state`) and formats them into a single, neat summary for the user.

This pattern lets us get a lot of work done, fast! 🚀

```
+-------------------------------+
|  parallel_planner_agent ⚡     |
| SequentialAgent:              |
| 1. Run parallel research      |
| 2. Synthesize results         |
+---------------+---------------+
                |
     +----------+----------------------+
     |                                 |
     v                                 v
+-------------------------+       +-----------------------------+
| parallel_research_agent ⚡   |   | synthesis_agent 📋          |
| ParallelAgent:              |   | Combine results            |
| - museum_finder_agent 🖼️     |   | Output: Bulleted summary   |
| - concert_finder_agent 🎵    |   +-----------------------------+
| - restaurant_finder_agent 🍽️ |
+-------------------------+

Final Output:
• Museum: XYZ  
• Concert: Artist at Venue  
• Restaurant: ABC
```

In [ ]:
# --- Agent Definitions for a Parallel Workflow ---

# Specialist Agent 1
museum_finder_agent = Agent(
    name="museum_finder_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction="You are a museum expert. Find the best museum based on the user's query. Output only the museum's name.",
    output_key="museum_result"
)

# Specialist Agent 2
concert_finder_agent = Agent(
    name="concert_finder_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction="You are an events guide. Find a concert based on the user's query. Output only the concert name and artist.",
    output_key="concert_result"
)

# We can reuse our foodie_agent for the third parallel task!
# Just need to give it a new output_key for this workflow.
# restaurant_finder_agent = foodie_agent.copy(update={"output_key": "restaurant_result"})
restaurant_finder_agent = Agent(
    name="restaurant_finder_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are an expert food critic. Your goal is to find the best restaurant based on a user's request.

    When you recommend a place, you must output *only* the name of the establishment.
    For example, if the best sushi is at 'Jin Sho', you should output only: Jin Sho
    """,
    output_key="restaurant_result" # Set the correct output key for this workflow
)


# ✨ The ParallelAgent runs all three specialists at once ✨
parallel_research_agent = ParallelAgent(
    name="parallel_research_agent",
    sub_agents=[museum_finder_agent, concert_finder_agent, restaurant_finder_agent]
)

# Agent to synthesize the parallel results
synthesis_agent = Agent(
    name="synthesis_agent", model="gemini-2.5-flash",
    instruction="""You are a helpful assistant. Combine the following research results into a clear, bulleted list for the user.
    - Museum: {museum_result}
    - Concert: {concert_result}
    - Restaurant: {restaurant_result}
    """
)

# ✨ The SequentialAgent runs the parallel search, then the synthesis ✨
parallel_planner_agent = SequentialAgent(
    name="parallel_planner_agent",
    sub_agents=[parallel_research_agent, synthesis_agent],
    description="A workflow that finds multiple things in parallel and then summarizes the results."
)

print("🤖 Agent team supercharged with a ParallelAgent workflow!")

---
### Final Step: Updating the Router and Running the App

Now we just have one last thing to do: make our `router_agent` aware of these powerful new workflows! We'll add `iterative_planner_agent` and `parallel_planner_agent` to its list of available options.

Then we can run our app with new queries designed to trigger these advanced, multi-agent workflows.

```
                    +---------------------+
                    |    User Query 🗣️     |
                    +----------+----------+
                               |
                               v
                    +---------------------+
                    |   Router Agent 🤖    |
                    |  (Classify Request) |
                    +----------+----------+
                               |
      +-----------+-----------+-----------+-----------+------------+
      |           |           |           |           |            |
      v           v           v           v           v            v
+-------------+  +------------------+  +------------------+  +------------------+  +-----------------+
| foodie_agent|  | find_and_navigate|  | iterative_planner|  | parallel_planner |  | day_trip_agent  |
| 🍣 Food Only |  | 🧭 Seq Workflow   |  | 🔁 Loop Workflow  |  | ⚡ Parallel Tasks |  | 🧳 Basic Plan     |
+-------------+  +------------------+  +------------------+  +------------------+  +-----------------+
```

In [ ]:
# --- The ULTIMATE Router Agent --- #

router_agent = Agent(
    name="router_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a master request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
    Do not answer the query yourself, only return the name of the most appropriate choice.

    Available Options:
    - 'foodie_agent': For queries *only* about finding a single food place.
    - 'find_and_navigate_agent': For queries that ask to *first find a place* and *then get directions* to it.
    - 'iterative_planner_agent': For planning a trip with a specific constraint that needs checking, like travel time.
    - 'parallel_planner_agent': For queries that ask to find multiple, independent things at once (e.g., a museum AND a concert AND a restaurant).
    - 'day_trip_agent': A general planner for any other simple day trip requests.

    Only return the single, most appropriate option's name and nothing else.
    """
)

# The master dictionary of all our executable agents and workflows
worker_agents = {
    "day_trip_agent": day_trip_agent,
    "foodie_agent": foodie_agent, # For simple food queries
    "find_and_navigate_agent": find_and_navigate_agent, # Sequential
    "iterative_planner_agent": iterative_planner_agent, # Loop
    "parallel_planner_agent": parallel_planner_agent,   # Parallel
}

# --- Let's Test Everything! ---

async def run_fully_loaded_app():
    queries = [
        # Test Case 1: Simple Sequential Flow
        "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station.",

        # Test Case 2: Iterative Loop Flow
        "Plan me a day in San Francisco with a museum and a nice dinner, but make sure the travel time between them is very short.",

        # Test Case 3: Parallel Flow
        "Help me plan a trip to SF. I need one museum, one concert, and one great restaurant."
    ]

    for query in queries:
        print(f"\n{'='*60}\n🗣️ Processing New Query: '{query}'\n{'='*60}")

        # 1. Ask the Router Agent to choose the right agent or workflow
        router_session = await session_service.create_session(app_name=router_agent.name, user_id=my_user_id)
        print("🧠 Asking the router agent to make a decision...")
        chosen_route = await run_agent_query(router_agent, query, router_session, my_user_id, is_router=True)
        chosen_route = chosen_route.strip().replace("'", "")
        print(f"🚦 Router has selected route: '{chosen_route}'")

        # 2. Execute the chosen route
        if chosen_route in worker_agents:
            worker_agent = worker_agents[chosen_route]
            print(f"--- Handing off to {worker_agent.name} ---")
            worker_session = await session_service.create_session(app_name=worker_agent.name, user_id=my_user_id)
            await run_agent_query(worker_agent, query, worker_session, my_user_id)
            print(f"--- {worker_agent.name} Complete ---")
        else:
            print(f"🚨 Error: Router chose an unknown route: '{chosen_route}'")

await run_fully_loaded_app()

---
## 🎉 Congratulations! 🎉

You've completed the Enhanced ADK Adventure! You have successfully. Let's review the advanced orchestration patterns you've successfully implemented:

- **The Router Pattern**: You built a master router agent capable of analyzing user intent and delegating tasks to the appropriate specialist agent or workflow.

- **Sequential Workflows**: Using SequentialAgent, you elegantly chained agents together, creating clean, readable code for multi-step tasks without manual data handling.

- **Iterative Refinement**: You constructed a sophisticated feedback loop with LoopAgent, enabling your agents to plan, self-critique, and improve their output until it met specific constraints.

- **Parallel Power**: You maximized speed and efficiency by using ParallelAgent to run multiple research tasks concurrently, later synthesizing the results into a unified response.


```
   __            /\_/\         /\_/\        /\_/\         __             (\__/)
o-''|\_____/).  ( o.o )       ( -.- )      ( ^_^ )     o-''|\_____/).    ( ^_^ )
 \_/|_)     )    > ^ <         > * <        >💖<         \_/|_)     )     / >🌸< \
    \  __  /                                              \  __  /         /   \
    (_/ (_/                                               (_/ (_/        (___|___)
```


# Task
Test the fully loaded multi-agent application by running various queries to observe how the router agent delegates tasks to sequential, iterative, and parallel workflows.

## Understand Agent's Task

### Subtask:
Clarify the specific task the fully loaded multi-agent application needs to perform, including how the router agent delegates tasks to sequential, iterative, and parallel workflows.


### Clarification of the Fully Loaded Multi-Agent Application's Task

The primary task of the fully loaded multi-agent application is to intelligently process a user's natural language query and delegate it to the most appropriate specialized agent or multi-agent workflow for execution. This delegation is handled by the `router_agent`, which acts as the central orchestrator.

#### Router Agent's Delegation Logic

The `router_agent`'s instruction explicitly defines its role:

```
You are a master request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
Do not answer the query yourself, only return the name of the most appropriate choice.

Available Options:
- 'foodie_agent': For queries *only* about finding a single food place.
- 'find_and_navigate_agent': For queries that ask to *first find a place* and *then get directions* to it.
- 'iterative_planner_agent': For planning a trip with a specific constraint that needs checking, like travel time.
- 'parallel_planner_agent': For queries that ask to find multiple, independent things at once (e.g., a museum AND a concert AND a restaurant).
- 'day_trip_agent': A general planner for any other simple day trip requests.
```

This instruction guides the `router_agent` to classify incoming queries into one of the predefined categories, each corresponding to a specialist agent or a complex multi-agent workflow.

#### Multi-Agent Workflows and Their Purposes:

1.  **`foodie_agent`**: Handles simple queries solely focused on finding a single food place. It uses `google_search` and saves its recommendation (restaurant name) to `state['destination']` for potential future use if part of a sequential workflow.

2.  **`day_trip_agent`**: A general-purpose planner for simple day trip requests that don't involve complex constraints or multiple independent searches. It also uses `google_search`.

3.  **`find_and_navigate_agent` (Sequential Workflow)**:
    *   **Structure**: This is a `SequentialAgent` comprising `foodie_agent` followed by `transportation_agent`.
    *   **Purpose**: Designed for queries that require *first finding a location* (e.g., a restaurant) and *then obtaining directions* to it. The `foodie_agent` identifies the destination and stores it in `state['destination']`. The `transportation_agent` then automatically uses this `destination` from the shared state to provide directions, demonstrating seamless data passing between agents in a sequence.

4.  **`iterative_planner_agent` (Loop Workflow)**:
    *   **Structure**: This is a `SequentialAgent` that first runs `planner_agent` and then enters a `LoopAgent` called `refinement_loop` (which contains `critic_agent` and `refiner_agent`).
    *   **Purpose**: Addresses queries that involve planning with specific constraints (e.g., short travel time between locations) that might require refinement. The `planner_agent` proposes an initial plan. The `refinement_loop` then iteratively checks this plan using the `critic_agent` (which might use `google_search` to check travel times) and refines it with the `refiner_agent` until the constraint is met or `max_iterations` is reached. The `refiner_agent` uses an `exit_loop` tool to terminate the loop when the plan is deemed feasible.

5.  **`parallel_planner_agent` (Parallel Workflow)**:
    *   **Structure**: This is a `SequentialAgent` that first runs `parallel_research_agent` (a `ParallelAgent`) and then `synthesis_agent`.
    *   **Purpose**: Optimizes queries asking for *multiple independent things* simultaneously (e.g., a museum, a concert, and a restaurant). The `parallel_research_agent` concurrently executes `museum_finder_agent`, `concert_finder_agent`, and `restaurant_finder_agent`. Each specialist agent saves its result to a unique key in the shared state (e.g., `museum_result`, `concert_result`, `restaurant_result`). Once all parallel tasks are complete, the `synthesis_agent` collects these results from the shared state and compiles them into a unified summary for the user.

#### Delegation Prediction for Test Queries:

Based on the router's instructions and the workflow descriptions, here's how the test queries in `run_fully_loaded_app` will be delegated:

1.  **Query**: "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station."
    *   **Prediction**: `find_and_navigate_agent`
    *   **Reasoning**: This query explicitly asks to *first find a place* and *then get directions*, directly matching the `find_and_navigate_agent`'s description.

2.  **Query**: "Plan me a day in San Francisco with a museum and a nice dinner, but make sure the travel time between them is very short."
    *   **Prediction**: `iterative_planner_agent`
    *   **Reasoning**: The inclusion of a specific constraint ("make sure the travel time between them is very short") that needs checking and potential refinement aligns perfectly with the `iterative_planner_agent`'s purpose.

3.  **Query**: "Help me plan a trip to SF. I need one museum, one concert, and one great restaurant."
    *   **Prediction**: `parallel_planner_agent`
    *   **Reasoning**: This query requests *multiple, independent things* (museum, concert, restaurant) simultaneously, which is the exact scenario the `parallel_planner_agent` is designed to handle efficiently.

## Test the fully loaded multi-agent application

### Subtask:
Run the `run_fully_loaded_app()` function to test the router agent's delegation and the execution of sequential, iterative, and parallel workflows.


**Reasoning**:
Execute the `run_fully_loaded_app()` function to test the various multi-agent workflows as instructed.



In [ ]:
await run_fully_loaded_app()